# Project: NIP
# Subroject: Sylhet Floods, June 2022
## Script: fracArea_timeSeries_UNOSAT.ipynb
This script generates a time series of the fractional inundated area, from the outputs of the UNOSAT surface water product.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
from itertools import chain
from pyproj import Transformer
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from datetime import datetime
import geopandas as gpd
import copy
import collections
from shapely.geometry import mapping
import pycrs
from shapely.geometry import Polygon
from functools import reduce

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Add path for the Helpers modules
module_path = os.path.abspath(os.path.join('C:/Users/alexsaunders/Documents/01_uoa/04_git/NIP/Sylhet/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import module from analyse_modifiedDevries containing helpful functions to use
import importlib
import Helpers.analyse_modifiedDevries as analyse_DV
importlib.reload(analyse_DV)
import Helpers.prepare_flood_raster as prep_raster
importlib.reload(prep_raster)

<module 'Helpers.prepare_flood_raster' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\prepare_flood_raster.py'>

## PART 1: Define the ROI(s) for computing fractional flooded area
Load in the admin areas and gauge locations, create some buffer sizes around the gauges to use as ROI for computing FFA

In [6]:
UNOSATPath = rootPath/'Raster/SylhetUNOSAT'
mosaics = [file for file in (UNOSATPath).iterdir() if file.is_file() and 'allTouchTrue' in str(file) and '20220525_W' in str(file)]
mosaics

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetUNOSAT/S1_20220525_WaterExtent_NE_allTouchTrue.tif')]

In [7]:
# Import the ROI geometry from shapefile
adminShapePath = rootPath/'Shapefiles/AdminHDX'
districts = gpd.read_file(adminShapePath/'bgd_admbnda_adm2_bbs_20201113.shp')
upazillas = gpd.read_file(adminShapePath/'bgd_admbnda_adm3_bbs_20201113.shp')
sylhet_dist = districts[districts['ADM2_EN']=='Sylhet']
sylhet_upa = upazillas[upazillas['ADM3_EN']=='Sylhet Sadar']

In [8]:
# Or instead create the ROI as a buffer around the gauge locations

# Import the gauge station locations
gaugePath = rootPath/'Stream_Gauge/StreamGaugeShapefiles'
gaugeLocs = gpd.read_file(gaugePath/'streamgaugew_division.shp') # read the file as a geopandas

# Create buffer around all gauges - use several different buffer sizes starting from 100 m, to 1 km
gaugeLocsGeoms = gaugeLocs.geometry
buffer_sizes = [5, 10, 15]#[0.1, 0.2, 0.5, 1]
pd.options.mode.chained_assignment = None  # default='warn'

for buffer_size in buffer_sizes:
    
    # Add new field to locs geodataframe to store the buffer geometries
    gaugeLocs['buffer_'+str(buffer_size)] = 0

    # For each gaugeLoc, get the buffer and add back to the geodataframe
    for i in range(0, len(gaugeLocs)):
        gaugeLocGeom = gaugeLocsGeoms[i]
        buffer = analyse_DV.geodesic_point_buffer(gaugeLocGeom.y, gaugeLocGeom.x, buffer_size) # lat, lon, buffer (in km)
        gaugeLocs['buffer_'+str(buffer_size)][i] = Polygon(buffer)

In [9]:
# Define the gauges in Sylhet District
streamGaugeDataPath = rootPath/'Stream_Gauge/'
gauges_in_sylhet = pd.read_csv(streamGaugeDataPath/'streamgauges_SylhetDist.csv', header=0)
gauges_in_sylhet = list(gauges_in_sylhet['streamvect'])
gaugeDataSummaryMaster = pd.read_csv(streamGaugeDataPath/'CombinedAll2004-2022/Combined/summary.csv', header=0)
gauges = [match for match in gauges_in_sylhet if match in list(gaugeDataSummaryMaster['stationID'][np.logical_and(~np.isnan(gaugeDataSummaryMaster['danger_level']),gaugeDataSummaryMaster['danger_level']<1000)])]
print(gauges)
gauges_in_sylhet_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges_in_sylhet)]
gauges_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges)]

['SW172.5', 'SW173', 'SW175.5', 'SW251', 'SW266', 'SW267']


In [10]:
# Choose which gauge to use as the ROI for FFA time series
#SW267 is Sylhet
#SW269 is Sunamganj
#gauge_sylhet = gaugeLocs[gaugeLocs['streamvect']=='SW267']
#gauge_sunamganj = gaugeLocs[gaugeLocs['streamvect']=='SW269']

In [11]:
gauges_list = list(chain.from_iterable([list(gauges_geom['streamvect']+'_'+str(buffer_size)+'km') for buffer_size in buffer_sizes]))
gauges_geom_list = list(chain.from_iterable([list(gauges_geom['buffer_'+str(buffer_size)]) for buffer_size in buffer_sizes]))

In [12]:
# Create a single geodataframe of ROIs over which to calculate FFA
rois = {'id': ['Sylhet_District', 'Sylhet_Upazilla']+gauges_list, 
        'geometry': list(sylhet_dist.geometry)+list(sylhet_upa.geometry)+gauges_geom_list}
rois_gdf = gpd.GeoDataFrame(rois, geometry='geometry', crs='epsg:4326')

## PART 2: Compute the fractional inundated area at each timestep
For each date, extract the fractional inundated area for the specified ROI.

In [13]:
mosaics

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetUNOSAT/S1_20220525_WaterExtent_NE_allTouchTrue.tif')]

In [14]:
# Get the dates of images
dates = ['20220525']
print(len(dates), 'dates:', dates)

1 dates: ['20220525']


In [11]:
# Run the function to create temp raster file for using in masking - ONLY NEEDS TO BE RUN ONCE

# Run the function which creates a raster of values 10, the same dimensions as the flood raster, to be used for clipping to the ROI later
# date1 = '20220604'
# mosaic = [mosaic for mosaic in mosaics if date1 in str(mosaic)][0]
# analyse_DV.create_temp_raster_for_mask(mosaic, rootPath, inputPath) # creates raster_tmp

In [15]:
# Load the temp raster in and intersect it with the ROI - this will be used for clipping to the ROI since it contains values of 10
# Also load the Devries raster, which will be used to reproject and match the GFM raster, to make the rest of the processing workflow the same
DevriesPath = rootPath/'Raster/Sylhet/Sen1MitchellSingleOrbit/Mosaic'
raster_tmp = rasterio.open(DevriesPath/'temp.tif') # can recreate this file if needs for a different ROI area by rerunning the above function
Devries_mosaics = [file for file in (DevriesPath).iterdir() if file.is_file() and '2022' in str(file)]
#raster_Devries = rasterio.open(Devries_mosaics[0])
raster_Devries = prep_raster.load_raster_for_comparison(Devries_mosaics[0], -20) # raster to match projection and extent, value to set not valid values

In [80]:
# Run the function to get the fractional flooded area for all image dates, for all the provided ROIs
# note that raster_tmp will need to be recreated if using images with a different extent (but otherwise does not need to be recreated)
# fracFloodArea_sylhetDist = analyse_DV.get_fracArea_timeSeries_GFM(mosaics, raster_tmp, raster_Devries, sylhet_dist)
# fracFloodArea_sylhetUpa = analyse_DV.get_fracArea_timeSeries_GFM(mosaics, raster_tmp, raster_Devries, sylhet_upa)


In [20]:
# Run the function to get the fractional flooded area for all image dates, for all the provided ROIs
mosaic_dates = []
pctWaters = []
rois_list = []

# Loop through image dates
for n, mosaic in enumerate(mosaics):
    
    # Get the date
    mosaic_date = dates[n]
    print('Mosaic:', mosaic_date, mosaic)

    # Load the raster using rioxarray
    raster = prep_raster.load_raster_for_comparison(mosaic, -10)

    # Reproject and match the GFM raster to the modified Devries raster to make the processing the same
    raster_reproj = prep_raster.reproj_match_raster(raster, raster_Devries)

    # Set the water values to 1
    water = copy.copy(raster_reproj)
    water.values[water.values==1]=1
    water.values[water.values!=1]=0
    
    # Loop through all ROI geometries
    for j in range(len(rois_gdf)):

        # Run the function to create the raster mask for the ROI, using the temp raster containing all values 10
        roiID = rois_gdf.id[j]
        roi = rois_gdf.loc[[j]]
        roi_mask = analyse_DV.create_roi_mask(raster_tmp, roi)

        # Apply the mask to the water_sum raster by performing an array sum operation
        water_roi = np.add(water, roi_mask)

        # Get the number of pixels of water for each date for the ROI only (pixel values in ROI have been shifted by +10 from original values)
        water_roi_vals = list(chain.from_iterable(list(chain.from_iterable(water_roi.values.tolist()))))
        counter_roi = collections.Counter(water_roi_vals) # 10 = none, 11 = water, other = invalid or outside ROI
        pctWater_roi = counter_roi[11] / (counter_roi[10] + counter_roi[11])
        print(counter_roi)
        print('ROI % water at {0} for {1}: {2:2.1f}%'.format(mosaic_date, roiID, pctWater_roi*100))
        
        # Record the results for the date and ROI
        mosaic_dates.append(mosaic_date)
        rois_list.append(roiID)
        pctWaters.append(pctWater_roi)

Mosaic: 20220525 Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetUNOSAT\S1_20220525_WaterExtent_NE_allTouchTrue.tif
Counter({0.0: 27936043, 11.0: 17547711, 10.0: 16613392, 1.0: 5111349})
ROI % water at 20220525 for Sylhet_District: 51.4%
Counter({0.0: 42592958, 1.0: 21439997, 10.0: 1956477, 11.0: 1219063})
ROI % water at 20220525 for Sylhet_Upazilla: 38.4%


KeyboardInterrupt: 

In [17]:
# Create dataframe of the results - fractional flooded area
fracFloodArea = pd.DataFrame(data = [mosaic_dates, rois_list, pctWaters], index = ['Date','ROI','FFA']).T
fracFloodArea = fracFloodArea.pivot(index='Date', columns='ROI', values='FFA')
fracFloodArea.index = pd.to_datetime(fracFloodArea.index)

In [19]:
# Export the results as csv
outputPath = rootPath/'Table/SylhetFracFloodedArea'
outputPath.mkdir(exist_ok=True)
pd.DataFrame.to_csv(fracFloodArea, outputPath/'FFA_sylhet_allROI_allGauges_UNOSATallTouchTrue_25.csv', index=True)

In [132]:
# If required, read back in from csv table
# fracFloodArea = pd.read_csv(rootPath/'Table/SylhetFracFloodedArea/FFA_sylhet_allROI_allGauges_GFM.csv', index_col='Date')
# fracFloodArea.index = pd.to_datetime(fracFloodAreaBuffers.index)